In [1]:
import cv2
import numpy as np
import mediapipe as mp
import pandas as pd
import matplotlib.pyplot as plt
from cvzone.HandTrackingModule import HandDetector
import time
from tensorflow.keras.models import load_model

In [2]:
detector = HandDetector(maxHands=1)

In [3]:
model = load_model('Models/oneData1.h5')

In [4]:
nums = ["Zero", "one", "Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine", "Ten", "Unknown"]

In [5]:
def number_to_string(num):
    return nums[num]

In [6]:
def pre_prediction(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = cv2.resize(gray, (64,64))
    img = img.reshape((1,64,64,1)) / 255.0
    p = model.predict(img)[0]
    return number_to_string(p.argmax())

In [7]:
# def preProcessing(img):
#     gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     img = cv2.resize(gray, (64,64))
#     pix = []
#     for i in range(64):
#         for j in range(64):
#             pix.append(img[i,j])
#     df = pd.DataFrame(pix).T
#     cols = [i for i in range(4096)]
#     df.columns = cols
    
#     return df.values

In [8]:
# def prediction(img):
#     img = img.reshape((1,64,64,1)) / 255.0
#     pred = np.array(model.predict(img)[0])
#     letter_prediction_dict = {nums[i]: pred[i] for i in range(len(nums))}
    
#     predOrdered = sorted(pred)
    
#     high1 = predOrdered[0]
#     high2 = predOrdered[1]
#     high3 = predOrdered[2]
    
#     for key,value in letter_prediction_dict.items():
#         if value==high1:
#                 print("Predicted Number 1: ", key)
#                 print('Confidence 1: ', 100*value)
#         elif value==high2:
#                 print("Predicted Number 2: ", key)
#                 print('Confidence 2: ', 100*value)
#         elif value==high3:
#                 print("Predicted Number 3: ", key)
#                 print('Confidence 3: ', 100*value)
# #     return letter_prediction_dict
# #     return np.argmax(pred[0])

In [10]:
idx = 0
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1)
mp_draw = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)
# fourcc = cv2.VideoWriter_fourcc(*"XVID")
# out = cv2.VideoWriter("Numbers_Project.mp4", fourcc,  15,(640,480))
mayhand = 0
while  True:
    ret,frame = cap.read()
    num = None
#     frame = cv2.flip(frame,1)
#     frame  = cv2.resize(frame, (1280,720))
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    _, frame = detector.findHands(frame)
    
    results = hands.process(rgb)
    
    h,w,_ = frame.shape
    
    if results.multi_hand_landmarks is not None:
        
        for hand in results.multi_hand_landmarks:
            
            x_max, y_max  = 0, 0
            x_min, y_min = w, h
            
            for i, lm in enumerate(hand.landmark):
                
                h,w, _ = frame.shape
                x,y = lm.x, lm.y
                cx,cy = int(x*w), int(y*h)
                
                if cx > x_max:
                    x_max = cx 
                if cy > y_max:
                    y_max = cy 
                if cx < x_min:
                    x_min = cx 
                if cy < y_min:
                    y_min = cy 
                    
                    
            
            x_max,x_min,y_max,y_min = x_max+20,x_min-20,y_max+20, y_min-20
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0,0,255), 3)

            roi = rgb[y_min:y_max, x_min:x_max]
            gr = cv2.cvtColor(roi, cv2.COLOR_RGB2GRAY)
            et,th = cv2.threshold(gr,50,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
#             th = cv2.adaptiveThreshold(gr, 255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 201,5)
            cv2.imshow("TH", th)
            
#             cv2.imwrite("Captured_data/"+str(idx)+".jpg", roi)
#             idx += 1
            myhand = roi

            mp_draw.draw_landmarks(frame, hand, mp_hands.HAND_CONNECTIONS)
    
            num = pre_prediction(roi)
            
            
            
#     out.write(frame)

    if num is None:
        Text = "Number : None"
    else:
        Text = "Number: " + str(num)
        
    cv2.putText(frame, Text, (200 ,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 5)
    
    cv2.imshow("Hand detection", frame)
    
    if cv2.waitKey(30) == 27:
        break
        
        
cv2.destroyAllWindows()
cap.release()
# out.release()

1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 31ms/step
